In [1]:
pip install langchain langchain-community langchain-openai langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 15.1 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 13.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.8/999.8 kB 15.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 12.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 15.9 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.0/802.0 kB 15.3 MB/s  0:00:00
   ━━━

In [2]:
pip install langchain-ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain-ollama]
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# For better security, load environment variables from a .env file
# from dotenv import load_dotenv
# load_dotenv()
# Make sure your OPENAI_API_KEY is set in the .env file

# Initialize the Language Model (using ChatOpenAI is recommended)
llm = ChatOpenAI(temperature=0)

# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
   "Extract the technical specifications from the following text:\n\n{text_input}"
)

# --- Prompt 2: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_template(
   "Transform the following specifications into a JSON object with 'cpu', 'memory', and 'storage' as keys:\n\n{specifications}"
)

# --- Build the Chain using LCEL ---
# The StrOutputParser() converts the LLM's message output to a simple string.
extraction_chain = prompt_extract | llm | StrOutputParser()

# The full chain passes the output of the extraction chain into the 'specifications'
# variable for the transformation prompt.
full_chain = (
   {"specifications": extraction_chain}
   | prompt_transform
   | llm
   | StrOutputParser()
)

# --- Run the Chain ---
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."

# Execute the chain with the input text dictionary.
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- Final JSON Output ---")
print(final_result)


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [6]:
MODEL_NAME = 'qwen3:latest'

In [7]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# No API key needed for Ollama running locally (defaults to http://localhost:11434)
# If your Ollama server is remote, set OLLAMA_BASE_URL, e.g.:
# os.environ["OLLAMA_BASE_URL"] = "http://your-server:11434"

# Initialize the local LLM via Ollama
llm = ChatOllama(model=MODEL_NAME, temperature=0)

# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_template(
   "Extract the technical specifications from the following text:\n\n{text_input}"
)

# --- Prompt 2: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_template(
   "Transform the following specifications into a JSON object with 'cpu', 'memory', and 'storage' as keys:\n\n{specifications}"
)

# --- Build the Chain using LCEL ---
extraction_chain = prompt_extract | llm | StrOutputParser()

full_chain = (
   {"specifications": extraction_chain}
   | prompt_transform
   | llm
   | StrOutputParser()
)

# --- Run the Chain ---
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- Final JSON Output ---")
print(final_result)



--- Final JSON Output ---
<think>
Okay, the user wants me to transform the given specifications into a JSON object with 'cpu', 'memory', and 'storage' as keys. Let me start by parsing the original text.

The text says: "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD." 

First, I need to identify each component. The processor is mentioned first: "3.5 GHz octa-core processor." So the CPU key should include both the speed (3.5 GHz) and the core count (octa-core, which is 8 cores). 

Next, the memory is "16GB of RAM." That's straightforward—16 gigabytes of RAM. The key is 'memory', and the value should be 16 GB. 

For storage, it's "1TB NVMe SSD." The storage type is an SSD, and it uses NVMe technology. The capacity is 1TB. So the 'storage' key should include the capacity (1TB) and the type (NVMe SSD). 

Wait, should I separate the interface (NVMe) from the storage type (SSD)? The original text combines them as "NVMe SSD," so it's safe to incl

In [8]:
final_result = full_chain.invoke({"text_input": input_text})


In [9]:
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."
prompt_extract = ChatPromptTemplate.from_template(
   "Extract the technical specifications from the following text:\n\n{text_input}"
)

In [10]:
extraction_chain = prompt_extract | llm | StrOutputParser()
extraction_chain.invoke({"text_input": input_text})

'<think>\nOkay, the user wants me to extract technical specifications from the given text. Let me read the text again: "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD." \n\nFirst, I need to identify each component mentioned. The first part is the processor. It says "3.5 GHz octa-core processor." So the processor speed is 3.5 GHz, and it\'s an octa-core, which means eight cores. I should note both the speed and the core count.\n\nNext is the RAM. The text states "16GB of RAM." That\'s straightforward—16 gigabytes of RAM. I need to make sure to include the unit (GB) and the quantity (16).\n\nThen there\'s the storage: "1TB NVMe SSD." The storage type here is an SSD, and it\'s NVMe, which is a type of interface. The capacity is 1TB. So I should list the storage type (SSD), the interface (NVMe), and the capacity (1TB).\n\nWait, should I separate NVMe as a separate specification or part of the SSD? The text mentions "NVMe SSD," so it\'s the type

In [11]:
# pip install langchain-ollama langchain-core

import re
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

# --- LLMs (same model, different configs) ---
# Plain responses (we'll still strip <think> just in case)

llm = ChatOllama(model=MODEL_NAME, temperature=0)
# Enforce strict JSON for the second step
llm_json = ChatOllama(model=MODEL_NAME, temperature=0, format="json")

# --- Helpers ---
def strip_think(text: str) -> str:
    # Removes any <think>...</think> blocks DeepSeek might include
    return re.sub(r"<think>.*?</think>\s*", "", text, flags=re.DOTALL | re.IGNORECASE)

stripper = RunnableLambda(strip_think)

# --- Prompt 1: Extract Information ---
prompt_extract = ChatPromptTemplate.from_messages([
    ("system", "You are a precise information extractor. Do NOT reveal your reasoning or chain-of-thought."),
    ("human", "Extract the technical specifications from the following text:\n\n{text_input}")
])

# --- Prompt 2: Transform to JSON ---
prompt_transform = ChatPromptTemplate.from_messages([
    ("system", "You convert data to STRICT JSON. Do NOT reveal reasoning. Output JSON only, no extra text."),
    ("human", "Transform the following specifications into a JSON object with keys exactly 'cpu', 'memory', and 'storage':\n\n{specifications}")
])

# --- Build Chains ---
extraction_chain = prompt_extract | llm | StrOutputParser() | stripper

full_chain = (
    {"specifications": extraction_chain}
    | prompt_transform
    | llm_json                 # format="json" enforces JSON-only output
    | StrOutputParser()
    | stripper
)

# --- Run ---
input_text = "The new laptop model features a 3.5 GHz octa-core processor, 16GB of RAM, and a 1TB NVMe SSD."
final_result = full_chain.invoke({"text_input": input_text})

print("\n--- Final JSON Output ---")
print(final_result)



--- Final JSON Output ---
{"cpu": "3.5 GHz octa-core", "memory": "16GB", "storage": "1TB NVMe SSD"}
